In [1]:
# import functions
import sys
import os
from pathlib import Path
import pandas as pd
import re
import numpy as np  
import submitit
import os 
import mne
from extraction_data import get_pid_eid_pairs
from config import paths
from Time_frequency_selectives import get_TFR, load_decoding_results, selective_channels , preprocess_LFP, split_trials


# get the pid and eid of the sessions to analyze
regionOfInterest = ['VISp'] # regions to include
pid_eid_pairs = get_pid_eid_pairs( regions = regionOfInterest,only_passive= True)

# get the decoding results
decoding_results_dir = os.path.join(str(Path(os.getcwd()).resolve().parent), 'Decoding_spikes', 'results')
results = load_decoding_results(pid_eid_pairs, suffix = 'v1_TPassive_TActive' , dir = decoding_results_dir)
right_selective, left_selective, right_sensitive, left_sensitive, neutral = selective_channels(results, p_value_threshold = 0.05, accuracy_threshold = 0.57)
print('Right selective: ', len(right_selective))
print('Left selective: ', len(left_selective))
RS_ch = right_selective['acronyms']
LS_ch = left_selective['acronyms']

# print the number of selective channels per region
data = []
for region in regionOfInterest:
    RS_count = len([ch for ch in RS_ch if re.match(rf'^{region}[12456]', ch)])
    LS_count = len([ch for ch in LS_ch if re.match(rf'^{region}[12456]', ch)])
    data.append([region, RS_count, LS_count])

# Create a DataFrame
df = pd.DataFrame(data, columns=['Region', 'Right Selective', 'Left Selective'])
print(df)

Connected to https://openalyx.internationalbrainlab.org as user "intbrainlab"
Total number of possible probe insertions: 73
Number of sessions without passive data: 23
Number of sessions with passive data: 50
Number of pids without decoding result: 6
 number of total channnels 1228
Right selective:  18
Left selective:  21
  Region  Right Selective  Left Selective
0   VISp               18              21


In [2]:
for pid, eid in pid_eid_pairs:
    Nch_index_right = len(right_selective.loc[right_selective['pid'] == pid, 'ch_indexs'].tolist())
    Nch_index_left = len(left_selective.loc[left_selective['pid'] == pid, 'ch_indexs'].tolist())
    if Nch_index_right > 0 or Nch_index_left > 0:
        print(f'PID: {pid}, Right Selective: {Nch_index_right}, Left Selective: {Nch_index_left}')


PID: b114e7e0-92b3-43bb-9529-53860e52606d, Right Selective: 1, Left Selective: 3
PID: d5e5311c-8beb-4f8f-b798-3e9bfa6bcdd8, Right Selective: 0, Left Selective: 2
PID: 27109779-1b15-4d74-893f-08216d231307, Right Selective: 0, Left Selective: 1
PID: 5544da75-8d34-47cb-9092-1f91e713e79c, Right Selective: 0, Left Selective: 1
PID: 799d899d-c398-4e81-abaf-1ef4b02d5475, Right Selective: 1, Left Selective: 0
PID: 12c0f3a9-518e-4dac-9651-5d95c2b4730f, Right Selective: 1, Left Selective: 1
PID: 27bac116-ea57-4512-ad35-714a62d259cd, Right Selective: 1, Left Selective: 1
PID: b83407f8-8220-46f9-9b90-a4c9f150c572, Right Selective: 1, Left Selective: 0
PID: 1f555fc6-5067-428b-a284-da6fb0b6b3a0, Right Selective: 4, Left Selective: 0
PID: 97207d87-3fcd-4ebb-b0c7-087bdbf8a95c, Right Selective: 0, Left Selective: 1
PID: b78b3c42-eee5-47c6-9717-743b78c0b721, Right Selective: 1, Left Selective: 0
PID: 4b93a168-0f3b-4124-88fa-a57046ca70e1, Right Selective: 1, Left Selective: 1
PID: 578ca001-8cf5-4d30-b58f